In [1]:
from random import randint
import numpy as np

In [4]:
class Dealer:
    flag_ace = False
    firstcard = 0
    cards = []
 
    def __init__(self, card):
        if card>10:
            card=10
        self.firstcard = card
 
    def getcard(self):
        card = randint(1,13)
        if card>10:
            card=10
        if card==1:
            self.flag_ace = True
        self.cards.append(card)
 
    def calculatescore(self):
        score = np.sum(np.array(self.cards))
        if self.flag_ace:
            if score<=11:
                score += 10    #This means one ace can be treat as 11 not 1
        return score
 
    def play(self):
        self.cards = [self.firstcard]
        if self.firstcard==1:
            self.flag_ace = True
        else:
            self.flag_ace = False
        while True:
            self.getcard()
            score = self.calculatescore()
            if self.flag_ace and score>17:
                break
            if self.flag_ace==False and score>16:
                break
        return score

In [5]:
class Player:
    cards = []
    flag_ace = False
 
    def __init__(self, card1, card2):
        self.cards = [card1, card2]
        if card1==1 or card2==1:
            self.flag_ace = True
 
    def hit(self):
        card = randint(1,13)
        if card>10:
            card=10
        if card==1:
            self.flag_ace = True
        self.cards.append(card)
 
    def calculatescore(self):
        score = np.sum(np.array(self.cards))
        if self.flag_ace:
            if score<=11:
                score += 10    #This means one ace can be treat as 11 not 1
        return score

In [ ]:
from random import randint
import numpy as np
policy = {}
policy['21'] = []
policy['A,10'] = []
for i in range(10):
    policy['21'].append('S')
    policy['A,10'].append('S')
for card1 in range(10, 1, -1):
    for card2 in range(10, 0, -1):
        if card2==1:
            key = 'A,'+str(card1)
        else:
            key = str(card1+card2)
        if key in policy:
            continue
        else:
            policy[key] = []
        for dealercard in range(1, 11):
            player1_money=10000
            player2_money=10000
            player3_money=10000
            bet = 100
            games = 100000
            player1_policy = 'S'
            player2_policy = 'D'  
            player3_policy = 'H'
 
            for i in range(games):
                #p = Player(player_card1,player_card2)
                p = Player(card1, card2)
                d = Dealer(dealercard)
 
                score1 = p.calculatescore()   #For player1, stand
                p.hit()
                score2 = p.calculatescore() 
                score3 = p.calculatescore()   #For player2 double, player3 hit
 
                while True:
                    if score3>=20:
                        break
                    if p.flag_ace:
                        cardsum = np.sum(np.array(p.cards))
                        if cardsum-1<11:
                            k = 'A,'+str(cardsum-1)
                        else:
                            k = str(score3)
                    else:
                        k = str(score3)
                    if policy[k][dealercard-1]=='S' or policy[k][dealercard-1]=='D/S':
                        break
                    else:
                        p.hit()
                        score3 = p.calculatescore()
 
                dealerscore = d.play()
 
                if score2>21:
                    if dealerscore>21:
                        player1_money += bet
                    else:
                        if score1>dealerscore:
                            player1_money += bet
                        if score1<dealerscore:
                            player1_money -= bet
                    player2_money -= 2*bet
                    player3_money -= bet
                else:
                    if score3>21:
                        player3_money -= bet
                        if dealerscore>21:
                            player1_money += bet
                            player2_money += 2*bet
                        else:
                            if score1>dealerscore:
                                player1_money += bet
                            if score1<dealerscore:
                                player1_money -= bet
                            if score2>dealerscore:
                                player2_money += 2*bet
                            if score2<dealerscore:
                                player2_money -= 2*bet
                            if dealerscore==21 and d.flag_ace==True and len(d.cards)==2:
                                if score2==21:
                                    player2_money -= 2*bet
                    else:    #score1<=21, score2<=21, score3<=21
                        if dealerscore>21:
                            player1_money += bet
                            player2_money += 2*bet
                            player3_money += bet
                        else:
                            if score1>dealerscore:
                                player1_money += bet
                            if score1<dealerscore:
                                player1_money -= bet
                            if score2>dealerscore:
                                player2_money += 2*bet
                            if score2<dealerscore:
                                player2_money -= 2*bet
                            if score3>dealerscore:
                                player3_money += bet
                            if score3<dealerscore:
                                player3_money -= bet
                            if dealerscore==21 and d.flag_ace==True and len(d.cards)==2:
                                if score2==21:
                                    player2_money -= 2*bet
                                if score3==21:
                                    player3_money -= bet
             
                del d,p
 
            max_money = max(player1_money, player2_money, player3_money)
            if max_money==player1_money:
                policy[key].append('S')
            if max_money==player2_money: 
                if player1_money>player3_money:
                    policy[key].append('D/S')
                else:
                    policy[key].append('D')
            if max_money==player3_money:
                policy[key].append('H')

In [ ]:
#调整policy中的Key的排序
keys = sorted(policy.keys())
sorted_keys = keys[12:18]
sorted_keys.extend(keys[:12])
sorted_keys.extend(keys[19:])
sorted_keys.append(keys[18])
with open('policy.csv', 'w') as f:
    policy_result = 'Player;2;3;4;5;6;7;8;9;10;A\n'
    for key in sorted_keys:
        policy_result += key+';'+';'.join(policy[key][1:])+';'+policy[key][0]+'\n'
    f.write(policy_result)
df_policy = pd.read_csv('policy.csv', header=0, index_col=0, sep=';')
df_policy.head(100)

In [ ]:
split_result = {}
for i in trange(2,10):
    split_result[str(i)+","+str(i)]=[]
    for c in trange(1, 11):   #Dealer card 2-10, A
        lose_counts = 0
        win_counts = 0
        draw_counts = 0
        split_gain = 0
        nonsplit_gain = 0
        for j in range(steps):
            for k in range(3):     #k-0,1, Split card, k-2, not split
                d = Dealer(c)
                while True:
                    secondcard = randint(1,13)
                    if secondcard>10:
                        secondcard = 10
                    if secondcard!=i:
                        break
                if k==2:
                    secondcard = i
                p = Player(i, secondcard)
                hit_count = 0
                player_lose = False
                doublescaler = 1
                while True:
                    score = p.calculatescore()
                    if score>18 and score<=21:
                        break
                    if score>21:
                        player_lose = True
                        break
                    if p.flag_ace:
                        cardsum = np.sum(np.array(p.cards))
                        if cardsum-1<11:
                            key = 'A,'+str(cardsum-1)
                        else:
                            key = str(score)
                    else:
                        key = str(score)
                    action = policy[key][c-1]
                    if hit_count==0: 
                        if action=='D' or action=='D/S':
                            doublescaler = 2
                    if (hit_count==0 and action=='D/S') or action=='D' or action=='H':
                        p.hit()
                        hit_count += 1
                    else:
                        score = p.calculatescore()
                        if score>21:
                            lose_counts += 1*doublescaler
                            if k<2:
                                split_gain -= 1*doublescaler
                            else:
                                nonsplit_gain -= 1*doublescaler
                            player_lose = True
                            break
                        else:
                            player_lose = False
                            break
                if player_lose==False:
                    dealerscore = d.play()
                    if dealerscore>21:
                        win_counts += 1*doublescaler
                        if k<2:
                            split_gain += 1*doublescaler
                        else:
                            nonsplit_gain += 1*doublescaler
                    else:
                        if score<dealerscore:
                            lose_counts += 1*doublescaler
                            if k<2:
                                split_gain -= 1*doublescaler
                            else:
                                nonsplit_gain -= 1*doublescaler
                        elif score>dealerscore:
                            win_counts += 1*doublescaler
                            if k<2:
                                split_gain += 1*doublescaler
                            else:
                                nonsplit_gain += 1*doublescaler
                        else:
                            draw_counts += 1
                else:
                    if k<2:
                        split_gain -= 1*doublescaler
                    else:
                        nonsplit_gain -= 1*doublescaler
                del d, p
        split_result[str(i)+","+str(i)].append([split_gain,nonsplit_gain])   
 
split_policy = {}
for key in split_result:
    split_policy[key] = []
    for i in range(10):
        if split_result[key][i][0]<split_result[key][i][1]:
            policykey = str(int(key.split(',')[0])*2)
            split_policy[key].append(policy[policykey][i])
        else:
            split_policy[key].append('P')
 
split_policy_result = 'Player;2;3;4;5;6;7;8;9;10;A\n'
for key in split_policy:
    split_policy_result += key+';'+';'.join(split_policy[key][1:])+';'+split_policy[key][0]+'\n'
with open('split_policy.csv', 'w') as f:
    f.write(split_policy_result)
df_split = pd.read_csv('split_policy.csv', header=0, index_col=0, sep=';')
df_split.head(100)

In [ ]:
with open('policy.csv', 'r') as f:
    lines = f.readlines()
with open('split_policy.csv', 'r') as f:
    lines.extend(f.readlines()[1:])
complete_policy = {}
for l in lines[1:]:
    a = l.strip().split(';')
    complete_policy[a[0]] = [a[-1]]
    complete_policy[a[0]].extend(a[1:])

In [ ]:
def play(self, dealercard, policy, simulate=False):
    card1 = self.cards[0]
    card2 = self.cards[1]
    hitcount = 0
    scale = 1.0
    count = 0
    while True:
        if self.flag_ace and len(self.cards)==2:
            if max(card1, card2)==10:
                scale=1.5
                score=21
                break
            else:
                key = "A,"+str(np.sum(np.array(self.cards))-1)
                action = policy[key][dealercard-1]
        else:
            score = self.calculatescore()
            if score>=18:
                break
            else:
                if self.flag_ace:
                    cardsum = np.sum(np.array(self.cards))
                    if cardsum<12:
                        key = 'A,'+str(cardsum-1)
                    else:
                        key = str(score)
                else:
                    key = str(score)
                action = policy[key][dealercard-1]
        if (action=='D' or action=='D/S') and hitcount==0:
            if simulate:
                self.hit(0)
            else:
                self.hit()
            score = self.calculatescore()
            scale = 2.0
            break
        elif action=='S' or (action=='D/S' and hitcount>0):
            score = self.calculatescore()
            break
        else:
            if simulate:
                self.hit(count)
            else:
                self.hit()
            score = self.calculatescore()
            if score>=18:
                break
            hitcount+=1
            count += 1
    return score, scale

In [ ]:
bet = 100
games = 10
loops = 1
debug = True
money = []
for j in trange(loops):
    totalmoney = 1000
    for i in range(games):
        if debug:
            print('Game {}:'.format(i))
        dealercard = randint(1,13)
        if dealercard>10:
            dealercard = 10
        d = Dealer(dealercard)
 
        card1 = randint(1,13)
        scores = []
        if card1>10:
            card1 = 10
        card2 = randint(1,13)
        if card2>10:
            card2 = 10
        action = ''
 
        if card1==card2 and card1!=10:
            if card1==1:
                action = 'P'
            else:
                key = str(card1)+','+str(card2)
                action = complete_policy[key][dealercard-1]
        if action=='P':
            split_times = 2
            while split_times>0:
                card = randint(1,13)
                card_count += 1
                if card>10:
                    card=10
                if card!=card1:
                    p_temp = Player(card1, card)
                    score, scale = p_temp.play(dealercard, complete_policy, False)
                    playercards = [str(a) for a in p_temp.cards]
                    if debug:
                        print('Player:'+','.join(playercards))
                    del p_temp
                    scores.append((score, scale))
                    split_times -= 1
                else:
                    split_times += 1
        else:
            p = Player(card1, card2)
            score, scale = p.play(dealercard, complete_policy, False)
            scores.append((score, scale))
            playercards = [str(a) for a in p.cards]
            if debug:
                print('Player:'+','.join(playercards))
            del p
 
        dealerscore = d.play()
        dealercards = [str(a) for a in d.cards]
        if debug:
            print('Dealer:'+','.join(dealercards))
 
        for item in scores:
            score, scale = item
            if score>21:
                totalmoney -= bet
            else:
                if score==21 and scale==1.5:
                    if dealerscore==21 and len(d.cards)==2:
                        continue
                    else:
                        totalmoney += bet*1.5
                else:
                    if dealerscore>21:
                        totalmoney += bet*scale
                    elif dealerscore==21 and len(d.cards)==2:
                        if score==21 and scale==1.5:
                            continue
                        else:
                            totalmoney -= bet*scale
                    else:
                        if score>dealerscore:
                            totalmoney += bet*scale
                        elif score<dealerscore:
                            totalmoney -= bet*scale
                        else:
                            continue
        del d
        if debug:
            print("TotalMoney:{}".format(totalmoney))
    money.append(totalmoney)
print(money)

In [ ]:
def hit(self, cardnum=-1):
    if cardnum>=0:
        card = self.cardpool[cardnum]
    else:
        card = randint(1,13)
    if card>10:
        card=10
    if card==1:
        self.flag_ace = True
    self.cards.append(card)

In [ ]:
bet = 100
games = 100
loops = 1000
debug = False
money = []
mypolicywins = 0
stdpolicywins = 0
policys = [standard_policy, complete_policy]
for j in trange(loops):
    totalmoney = [100000,100000]
    for i in range(games):
        if debug:
            print('Game {}:'.format(i))
        dealercard = randint(1,13)
        if dealercard>10:
            dealercard = 10
        d = Dealer(dealercard)
 
        card1 = randint(1,13)
        scores = [[],[]]
        if card1>10:
            card1 = 10
        card2 = randint(1,13)
        if card2>10:
            card2 = 10
        cardpool = []
        splitpool = []
        for cardpoolnum in range(200):
            cardpool.append(randint(1,13))
        for splitpoolnum in range(10):
            splitpool.append([])
            for a in range(20):
                splitpool[-1].append(randint(1,13))
        action = ''
        for policyid in range(2):
            if debug:
                if policyid==0:
                    print("Standardpolicy")
                else:
                    print("completepolicy")
            if card1==card2 and card1!=10:
                if card1==1:
                    action = 'P'
                else:
                    key = str(card1)+','+str(card2)
                    action = policys[policyid][key][dealercard-1]
            if action=='P':
                split_times = 2
                split_count = 0
                card_count = 0
                while split_times>0:
                    card = cardpool[card_count]
                    card_count += 1
                    if card>10:
                        card=10
                    if card!=card1:
                        p_temp = Player(card1, card)
                        p_temp.cardpool = splitpool[split_count]
                        split_count += 1
                        score, scale = p_temp.play(dealercard, policys[policyid], True)
                        playercards = [str(a) for a in p_temp.cards]
                        if debug:
                            print('Player:'+','.join(playercards))
                        del p_temp
                        scores[policyid].append((score, scale))
                        split_times -= 1
                    else:
                        split_times += 1
            else:
                p = Player(card1, card2)
                p.cardpool = cardpool
                score, scale = p.play(dealercard, policys[policyid], True)
                scores[policyid].append((score, scale))
                playercards = [str(a) for a in p.cards]
                if debug:
                    print('Player:'+','.join(playercards))
                del p
 
        dealerscore = d.play()
        dealercards = [str(a) for a in d.cards]
        if debug:
            print('Dealer:'+','.join(dealercards))
 
        for policyid in range(2):
            for item in scores[policyid]:
                score, scale = item
                if score>21:
                    totalmoney[policyid] -= bet
                else:
                    if score==21 and scale==1.5:
                        if dealerscore==21 and len(d.cards)==2:
                            continue
                        else:
                            totalmoney[policyid] += bet*1.5
                    else:
                        if dealerscore>21:
                            totalmoney[policyid] += bet*scale
                        elif dealerscore==21 and len(d.cards)==2:
                            if score==21 and scale==1.5:
                                continue
                            else:
                                totalmoney[policyid] -= bet*scale
                        else:
                            if score>dealerscore:
                                totalmoney[policyid] += bet*scale
                            elif score<dealerscore:
                                totalmoney[policyid] -= bet*scale
                            else:
                                continue
        del d
        if debug:
            print("TotalMoney:{}".format(totalmoney))
    if totalmoney[0]<totalmoney[1]:
        #print('My policy win')
        mypolicywins += 1
    if totalmoney[0]>totalmoney[1]:
        stdpolicywins += 1
        #print('Standard policy win')
    
print(mypolicywins)
print(stdpolicywins)